In [1]:
import folium
import html
import pandas as pd
import numpy as np
import re
from folium import plugins

In [2]:
arts = pd.read_csv('mta-arts.csv')

In [3]:
arts.head()

,agency,art_title,artist_fname,artist_lname,date,description,img,line,material,station_name,artist,wiki1,wiki2,lat,lon,mta_link
0,LIRR,Ghosts Series,Andrew,Leicester,1994,Andrew Leicester's {Ghost Series} consists of ...,http://web.mta.info/mta/aft/images/permart/Lei...,Penn Station,Terra cotta wall murals in five locations and ...,Penn Station,Andrew Leicester,https://en.wikipedia.org/wiki/Pennsylvania_Sta...,https://en.wikipedia.org/wiki/Pennsylvania_Sta...,40.750638,-73.993899,http://web.mta.info/mta/aft/permanentart/perma...
1,LIRR,Eclipsed Time,Maya,Lin,1994,In {Eclipsed Time} Maya Lin tries to get commu...,http://web.mta.info/mta/aft/images/permart/Lin...,Penn Station,"Sand-blasted glass, aluminum, stainless steel,...",Penn Station,Maya Lin,https://en.wikipedia.org/wiki/Pennsylvania_Sta...,https://en.wikipedia.org/wiki/Pennsylvania_Sta...,40.750638,-73.993899,http://web.mta.info/mta/aft/permanentart/perma...
2,LIRR,Overlook,Allan and Ellen,Wexler,2009,The new MTA Long Island Rail Road (LIRR) Atlan...,http://web.mta.info/mta/aft/images/permart/wex...,Babylon Branch,Granite,Atlantic Terminal,Allan and Ellen Wexler,https://en.wikipedia.org/wiki/Atlantic_Terminal,https://en.wikipedia.org/wiki/Atlantic_Terminal,40.684226,-73.977234,http://web.mta.info/mta/aft/permanentart/perma...
3,LIRR,"Morning Transit, Hempstead Plain & Evening Tra...",Roy,Nicholson,2002,"In the {Morning Transit, Hempstead Plain & Eve...",http://web.mta.info/mta/aft/images/permart/Nic...,Port Jefferson Branch,Glass mosaics on waiting room walls,Hicksville,Roy Nicholson,https://en.wikipedia.org/wiki/Hicksville_station,https://en.wikipedia.org/wiki/Hicksville_station,40.767101,-73.528686,http://web.mta.info/mta/aft/permanentart/perma...
4,LIRR,"For My Grandfather Noye Pride, a Locomotive En...",Joe,Zucker,1998,"At Huntington, Joe Zucker's 130-foot fantasy t...",http://web.mta.info/mta/aft/images/permart/Zuc...,Port Jefferson Branch,Faceted glass in windscreen,Huntington,Joe Zucker,https://en.wikipedia.org/wiki/Huntington_Stati...,https://en.wikipedia.org/wiki/Huntington_stati...,40.852692,-73.410639,http://web.mta.info/mta/aft/permanentart/perma...


In [4]:
arts.loc[arts.line == 'Port Jefferson and Ronkonkoma Branches', 'line'] = 'Port Jefferson Branch, Ronkonkoma Branch'
arts.fillna(value='nan', inplace=True)

In [5]:
# Get a list of all train lines
all_lines = ','.join(str(v) for v in arts.line.tolist()).split(',')
all_lines = [line.strip() for line in all_lines]
all_lines += all_lines[all_lines.index('L N Q R W 4 5')].split(' ')
all_lines.remove('L N Q R W 4 5')
all_lines = list(set(all_lines))                 

In [46]:
def create_icon(x):
    print('creating icon for: ' + x.art_title)
    links = x.img.split(', ')
    img = [link for link in links if '1s.jpg' in link or '1-s.jpg' in link][0]
    return folium.features.CustomIcon(icon_image=img)

def create_tooltip(x):
    img_links = x.img.split(', ')
    img = [link for link in img_links if '1m.jpg' in link or '1-m.jpg' in link][0]
    return '<img src="' + img + '"></img>'

def create_popup(x):
    table = x[['art_title', 'artist', 'date', 'agency','station_name', 'line', 'material', 'description']].transpose()
    table.description = re.sub('{|}', '', table.description)
    if not pd.isnull(x.mta_link):
        table.art_title = '<a target="_blank" href="' + x.mta_link + '">' + table.art_title + '</a>'
    table.index = ['Art Title:', 'Artist:', 'Year:', 'Agency:', 'Station:', 'Line:', 'Material:', 'Description:']
    table[table.index=='MTA Link'] = '<a href="' + table[table.index=='MTA Link'] + '">link</a>'
    with pd.option_context('display.max_colwidth', -1):
        table = table.to_frame().to_html(header=False, escape=False)
        table = table.replace('border="1"', 'border="0"')
        table = table.replace('<th>', '<th valign="top">')
    return folium.Popup(html=table, max_width=600)

def get_lines(x):
    lines = x.line.split(',')
    if x.line == 'L N Q R W 4 5':
        lines = x.line.split(' ')
    lines = [line.strip() for line in lines]
    return lines

def create_marker(x):
    lat, lon = x.lat, x.lon
    popup = create_popup(x)
    tooltip = create_tooltip(x)
    icon = folium.Icon(color='blue', icon='info-sign')
    return folium.Marker([lat, lon], popup=popup, tooltip=tooltip, icon=icon).add_to(agency_groups[x.agency])
#     lines = get_lines(x)
#     for line in lines:
#         line_groups[line].add_child(folium.Marker([lat, lon], popup=popup, tooltip=tooltip))

In [47]:
m = folium.Map(location=[40.7493, -73.6407], 
               zoom_start=11,
               tiles='cartodbdark_matter')

fg = folium.FeatureGroup(name='All', control=False)
m.add_child(fg)

agency_groups = {}
for agency in arts.agency.unique().tolist():
    agency_groups[agency] = plugins.FeatureGroupSubGroup(fg, agency)
    m.add_child(agency_groups[agency])

# line_groups = {}
# for line in all_lines:
#     line_groups[line] = plugins.FeatureGroupSubGroup(fg, line)
#     m.add_child(line_groups[line])

In [48]:
_ = arts.apply(create_marker, axis=1)

In [49]:
folium.LayerControl().add_to(m)

In [50]:
m

In [51]:
m.save('test.html')

In [16]:
help(folium.Icon)

Help on class Icon in module folium.map:

class Icon(branca.element.MacroElement)
 |  Creates an Icon object that will be rendered
 |  using Leaflet.awesome-markers.
 |  
 |  Parameters
 |  ----------
 |  color : str, default 'blue'
 |      The color of the marker. You can use:
 |  
 |          ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
 |           'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
 |           'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
 |           'gray', 'black', 'lightgray']
 |  
 |  icon_color : str, default 'white'
 |      The color of the drawing on the marker. You can use colors above,
 |      or an html color code.
 |  icon : str, default 'info-sign'
 |      The name of the marker sign.
 |      See Font-Awesome website to choose yours.
 |      Warning : depending on the icon you choose you may need to adapt
 |      the `prefix` as well.
 |  angle : int, default 0
 |      The icon will be rotated by this amount of degrees.
 